In [ ]:
# Temperature thresholds for heat stress calculations
TMaxOptimum = 32  # Optimal maximum daily temperature for plant growth (°C)
TMaxLimit = 38    # Upper limit of maximum daily temperature before severe stress (°C)
TMinOptimum = 22  # Optimal minimum daily temperature for plant growth (°C)
TMinLimit = 28    # Upper limit of minimum daily temperature before severe stress (°C)
TBase = 10        # Base temperature used for Growing Degree Days (GDD) calculation (°C)

# Formulas and set variables
GDD_FORMULA = 0.5 * (TMax + TMin) / 2 - TBase  # Formula to calculate Growing Degree Days (GDD)
GDD_OPTIMAL = (2000, 2500)  # Optimal GDD range required for crop growth
PRECIP_OPTIMAL = (1000, 1500)  # Optimal precipitation range in mm
PH_OPTIMAL = (5.5, 6.5)  # Optimal soil pH range for healthy plant growth
N_OPTIMAL = (0.051, 0.103)  # Optimal nitrogen availability in soil (g/kg)

def calculate_daytime_heat_stress(TMax, TMaxOptimum, TMaxLimit):
    """
    Calculate daytime heat stress based on maximum daily temperature.
    Returns a stress score from 0 (no stress) to 9 (severe stress).
    """
    if TMax <= TMaxOptimum:
        return 0  # No stress if temperature is within optimal range
    elif TMaxOptimum < TMax < TMaxLimit:
        return min(9, 9 * ((TMax - TMaxOptimum) / (TMaxLimit - TMaxOptimum)))  # Scale stress proportionally
    else:
        return 9  # Maximum stress if temperature exceeds limit

def calculate_nighttime_heat_stress(TMin, TMinOptimum, TMinLimit):
    """
    Calculate nighttime heat stress based on minimum daily temperature.
    Returns a stress score from 0 (no stress) to 9 (severe stress).
    """
    if TMin <= TMinOptimum:
        return 0  # No stress if temperature is within optimal range
    elif TMinOptimum < TMin < TMinLimit:
        return min(9, 9 * ((TMin - TMinOptimum) / (TMinLimit - TMinOptimum)))  # Scale stress proportionally
    else:
        return 9  # Maximum stress if temperature exceeds limit

def calculate_drought_index(cum_precip, cum_evap, soil_moisture, T_average):
    """
    Calculate drought risk index based on cumulative precipitation, evaporation, soil moisture, and temperature.
    Returns qualitative drought risk levels: 'No risk', 'Medium risk', or 'High risk'.
    """
    drought_index = (cum_precip - cum_evap) + soil_moisture / T_average  # Compute drought balance
    if drought_index > 0:
        return 'No risk'  # Sufficient moisture
    elif drought_index < 0:
        return 'High risk'  # Significant moisture deficit
    else:
        return 'Medium risk'  # Moderate risk level

     
def calculate_yield_risk(GDD, P, pH, N, GDD_opt=GDD_OPTIMAL, P_opt=PRECIP_OPTIMAL, 
                          pH_opt=PH_OPTIMAL, N_opt=N_OPTIMAL, 
                          w1=w1, w2=w2, w3=w3, w4=w4):
    """
    Calculate yield risk based on GDD, Precipitation, pH, and Nitrogen.
    
    Parameters:
    - GDD: Actual Growing Degree Days
    - P: Actual precipitation (mm)
    - pH: Actual soil pH
    - N: Actual available nitrogen (kg/ha)
    - GDD_opt, P_opt, pH_opt, N_opt: Optimal ranges
    - w1, w2, w3, w4: Weighting factors

    Returns:
    - Yield Risk Score (lower is better)
    """

    # Squared deviation from optimal values
    GDD_risk = ((GDD - sum(GDD_opt) / 2) ** 2) * w1
    P_risk = ((P - sum(P_opt) / 2) ** 2) * w2
    pH_risk = ((pH - sum(pH_opt) / 2) ** 2) * w3
    N_risk = ((N - sum(N_opt) / 2) ** 2) * w4

    # Total Yield Risk Score
    YR = GDD_risk + P_risk + pH_risk + N_risk

    return YR

In [7]:
# Load crops data from regions json file
import json

# Load the crops data from the json file
with open('Crops_per_state.json', 'r') as file:
    crops_data = json.load(file)

# Print the loaded data to verify
print(crops_data)


{'Andhra Pradesh': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[81.0769932018348, 16.914269930790695, 16], [81.07800601485741, 16.90898248759946, 16], [81.0791509339249, 16.90900355340476, 16], [81.08038392369082, 16.908708631919183, 16], [81.08118756880668, 16.908845559809066, 16], [81.08118756880668, 16.91037282568341, 16], [81.08258569113008, 16.91113118758753, 16], [81.08294898275625, 16.91310080772108, 16], [81.0821893729912, 16.915681294921256, 16], [81.0769932018348, 16.914269930790695, 16]]]}}, {'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[81.06533599714135, 16.905496745933732, 16], [81.06362538271367, 16.902739012845544, 16], [81.06681228082607, 16.901169553241076, 16], [81.06856976140307, 16.90377036490125, 16], [81.06533599714135, 16.905496745933732, 16]]]}}, {'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[81.

In [5]:
# transform regional crops data into dictionary state_crops of the following format:
# {
#     "state": [
#         [lon, lat, alt],
#         [lon, lat, alt],
#         ...
#     ]
# }
state_crops = dict()
for state in crops_data:
    state_crops[state] = []
    for feature in crops_data[state]['features']:
        sum_coordinates = [0, 0, 0]
        # Get the list of coordinates for the current feature
        coordinates = feature['geometry']['coordinates'][0]
        # print(coordinates)
        # Iterate through each coordinate in the list
        
        for coords in coordinates:
            if len(coords) == 2:
                lat, lon = coords
                alt = 0
                sum_coordinates[0] += lat
                sum_coordinates[1] += lon
                sum_coordinates[2] = ''
                state_crops[state].append([sum_coordinates[1]/len(coordinates), sum_coordinates[0]/len(coordinates), ''])

            else:
                lat, lon, alt = coords  
                # Add the values to the sum_coordinates
                sum_coordinates[0] += lat
                sum_coordinates[1] += lon
                sum_coordinates[2] += alt
            # Once we have ALS I should change the last 16 there and code above
                state_crops[state].append([sum_coordinates[1]/len(coordinates), sum_coordinates[0]/len(coordinates), sum_coordinates[2]/len(coordinates)])

# Print the average coordinates
print("State crops coordinates:", state_crops)

State crops coordinates: {'Andhra Pradesh': [[1.6914269930790695, 8.10769932018348, 1.6], [3.3823252418390153, 16.21549992166922, 3.2], [5.0732255971794915, 24.32341501506171, 4.8], [6.76409646037141, 32.431453407430794, 6.4], [8.454981016352317, 40.53957216431146, 8.0], [10.146018298920657, 48.64769092119213, 9.6], [11.83713141767941, 56.755949490305134, 11.2], [13.528441498451519, 64.86424438858076, 12.8], [15.220009627943645, 72.97246332587989, 14.4], [16.911436621022716, 81.08016264606336, 16.0], [3.3810993491867465, 16.21306719942827, 3.2], [6.761647151755855, 32.425792275971006, 6.4], [10.141881062404071, 48.63915473213622, 9.6], [13.52263513538432, 64.85286868441683, 12.8], [16.903734484571068, 81.0659358838451, 16.0], [2.412126693977062, 11.580849960754506, 2.2857142857142856], [4.824229971045442, 23.16169312575219, 4.571428571428571], [7.236331133849766, 34.74250314804123, 6.857142857142857], [9.648166955182598, 46.32330985605945, 9.142857142857142], [12.059974233651712, 57.90

In [6]:
# put them in a format that can be digested by weather API
for state in state_crops:
    for i, coord in enumerate(state_crops[state]):
        print(f"{coord[0]}, {coord[1]}, {coord[2]}, {state}_cropField#{i}")

1.6914269930790695, 8.10769932018348, 1.6, Andhra Pradesh_cropField#0
3.3823252418390153, 16.21549992166922, 3.2, Andhra Pradesh_cropField#1
5.0732255971794915, 24.32341501506171, 4.8, Andhra Pradesh_cropField#2
6.76409646037141, 32.431453407430794, 6.4, Andhra Pradesh_cropField#3
8.454981016352317, 40.53957216431146, 8.0, Andhra Pradesh_cropField#4
10.146018298920657, 48.64769092119213, 9.6, Andhra Pradesh_cropField#5
11.83713141767941, 56.755949490305134, 11.2, Andhra Pradesh_cropField#6
13.528441498451519, 64.86424438858076, 12.8, Andhra Pradesh_cropField#7
15.220009627943645, 72.97246332587989, 14.4, Andhra Pradesh_cropField#8
16.911436621022716, 81.08016264606336, 16.0, Andhra Pradesh_cropField#9
3.3810993491867465, 16.21306719942827, 3.2, Andhra Pradesh_cropField#10
6.761647151755855, 32.425792275971006, 6.4, Andhra Pradesh_cropField#11
10.141881062404071, 48.63915473213622, 9.6, Andhra Pradesh_cropField#12
13.52263513538432, 64.85286868441683, 12.8, Andhra Pradesh_cropField#13
1

In [25]:
Kharif_seasons = []
for year in range(2021, 2025):
    Kharif_seasons.append(f'{year}-05-01T+00:00/{year}-09-01T+00:00')
Kharif_seasons

['2021-05-01T+00:00/2021-09-01T+00:00',
 '2022-05-01T+00:00/2022-09-01T+00:00',
 '2023-05-01T+00:00/2023-09-01T+00:00',
 '2024-05-01T+00:00/2024-09-01T+00:00']

In [49]:
import pandas as pd
weather_df = pd.read_csv('weather_hist.csv')
static_df = pd.read_csv('static.csv')

In [50]:
weather_df.head()
# if variable is NEMSAUTO Temperature and aggregation is max, then we should get the max temperature from that row's columns 20210501T0000 through 20240901T0000
for row in weather_df.iterrows():
    if row['variable'] == 'NEMSAUTO Temperature' and row['aggregation'] == 'max':
         




,location,lat,lon,asl,variable,unit,level,timeResolution,aggregation,20210501T0000,...,20240823T0000,20240824T0000,20240825T0000,20240826T0000,20240827T0000,20240828T0000,20240829T0000,20240830T0000,20240831T0000,20240901T0000
0,Arunachal Pradesh_cropField#0,27.66654,95.625,122.559,NEMSAUTO Temperature,°C,sfc,daily,max,52.80,...,40.01,40.96,41.22,42.05,40.29,40.29,41.70,41.49,34.47,35.93
1,Arunachal Pradesh_cropField#1,27.66654,95.625,122.559,NEMSAUTO Temperature,°C,sfc,daily,max,52.80,...,40.01,40.96,41.22,42.05,40.29,40.29,41.70,41.49,34.47,35.93
2,Arunachal Pradesh_cropField#2,27.99988,95.625,233.573,NEMSAUTO Temperature,°C,sfc,daily,max,51.19,...,37.00,39.51,36.34,40.92,37.76,37.82,38.21,38.71,32.52,37.20
3,Arunachal Pradesh_cropField#3,27.99988,95.625,233.573,NEMSAUTO Temperature,°C,sfc,daily,max,51.19,...,37.00,39.51,36.34,40.92,37.76,37.82,38.21,38.71,32.52,37.20
4,Arunachal Pradesh_cropField#4,27.66654,95.625,122.559,NEMSAUTO Temperature,°C,sfc,daily,max,52.80,...,40.01,40.96,41.22,42.05,40.29,40.29,41.70,41.49,34.47,35.93


In [ ]:
# dataframe to store it all
import pandas as pd
df = pd.DataFrame(columns=['Field_ID',  
                           'season', 
                           'Tmax', 
                           'Tmin', 
                           'Tavg',
                           'T_base', # its same for every row, and we should grab it from gpt
                           'cumu_rainfall',
                           'rainfall_opt', 
                           'cumu_evap',
                           'avg_soil_moisture',
                           'GDD', 
                           'GDD_opt', 
                           'ph', 
                           'ph_opt', 
                           'N', 
                           'N_opt',
                        # ---These will be calculated with variables above------------- 
                           'DTHS',
                           'NTHS',
                           'DR',
                           'YR',
                        # ---Target variable -----------------------------------------
                           'yield_kg_per_ha'
                           ])


In [ ]:
Field_ID = [f"cropField#{i}" for i in range(len(avg_coords))]
season = Kharif_seasons


In [27]:
avg_coords

[[81.08016264606336, 16.911436621022716, 16.0],
 [81.0659358838451, 16.903734484571068, 16.0],
 [81.06621419807364, 16.88418956483609, 16.0],
 [81.05793705724372, 16.882190266835263, 16.0],
 [81.0680780862929, 16.887605046340376, 16.0],
 [81.0591467302026, 16.88764875954618, 16.0]]

# Get Max Temp

In [29]:
import requests
import json

# Your API key
api_key = "e4e4d60f7203"

# Define the endpoint URL
url = f"http://my.meteoblue.com/dataset/query?apikey={api_key}"

location_names = [f"cropField#{i}" for i in range(len(avg_coords))]

# Your query payload
payload = {
    "units": {
        "temperature": "C",
        "velocity": "km/h",
        "length": "metric",
        "energy": "watts"
    },
    "geometry": {
        "type": "MultiPoint",
        "coordinates": avg_coords,  # Use the coordinates directly
        "locationNames": location_names,  # Use the properly generated location names
        "excludeSeaPoints": True,
        "mode": "preferLandWithMatchingElevation"
    },
    "format": "json",
    "timeIntervals": [
        ""
    ],
    "timeIntervalsAlignment": "none",
    "queries": [
        {
            "domain": "NEMSAUTO",
            "gapFillDomain": None,
            "timeResolution": "daily",
            "codes": [
                {
                    "code": 11,
                    "level": "sfc",
                    "aggregation": ""
                }
            ]
        }
    ]
}

# Set headers
headers = {
    "Content-Type": "application/json"
}

# # Make the POST request
# response = requests.post(url, data=json.dumps(payload), headers=headers)

# # Print the response
# print(response.status_code)
# print(json.dumps(response.json(), indent=4))

# # If you want to save the response to a file
# with open("meteoblue_response.json", "w") as f:
#     json.dump(response.json(), f, indent=4)

In [31]:
print(payload["queries"][0]["codes"][0]["aggregation"])

max


In [ ]:
# Initialize the main data structure
data_collection = {}

# Iterate through each field and gather data for the years 2022, 2023, and 2024
for i, coord in enumerate(avg_coords):
    field_id = f"cropField#{i}"
    data_collection[field_id] = {
        "location": coord,
        "year": {}
    }
    
    for season in Kharif_seasons:  # Loop through the years 2022 to 2024
        # Update the time interval for the current year
        time_interval = season
        
        # Update the payload with the new time interval
        payload["timeIntervals"] = [time_interval]
        
        payload["queries"][0]["codes"][0]["aggregation"] = 'max'
        # Make the POST request
        response = requests.post(url, data=json.dumps(payload), headers=headers)
        
        # Extract max and min temperature from the response
        max_temp = response.json()[0]['codes'][0]['dataPerTimeInterval'][0]['data']  # Adjust based on actual response structure
        # Make the POST request
        payload["queries"][0]["codes"][0]["aggregation"] = 'min'
        # Make the POST request
        response = requests.post(url, data=json.dumps(payload), headers=headers)
        min_temp = response.json()[0]['codes'][0]['dataPerTimeInterval'][0]['data']  # Adjust based on actual response structure
        
        # Store the temperatures in the data structure
        data_collection[field_id]["year"][year] = {
            "max_temp": max_temp,
            "min_temp": min_temp
        }

# Print the collected data
print(data_collection)

TypeError: list indices must be integers or slices, not str